# Chapter 1: Code and Fix - The usual way

### Goals
- Implement a quick, working solution for the 1-D heat equation using explicit Euler scheme.
- Experience the "business-as-usual" approach to scientific computing.
- Observe the drawbacks of unstructured, monolithic code.

### Key Concepts
- Diffusion equation (PDE) and discretization
- Neumann boundary conditions (no-flux)
- Flux calculations and explicit Euler update
- "Code-and-fix" model: fast prototyping with minimal reasoning

---

## The 1-D Heat Equation

We start the way many scientific codes start: get something working.

Consider a 1-D insulated rod (no internal sources/sinks). The temperature u(x,t)
evolves by the heat equation:

$$\frac{\partial u}{\partial t} = \kappa \frac{\partial^2 u}{\partial x^2}$$

where $\kappa$ is the thermal diffusivity of the material [$m^2/s$].

## Discretization (finite-volume flavor)

 - Split the domain into $N$ cells of width $\Delta x$.
 - Let $u_i(t)$ be the average temperature in the i-th cell.
 - Diffusive flux of $u$ across an interface is (Fourier’s law for this scalar diffusion):

$$F_{i} = -\kappa \frac{u_{i} - u_{i-1}}{\Delta x}$$

where
 - $F_{i}$ is the heat flux at the interface between cells i-1 and i.
 - Positive $F$ is defined as flowing to the right.

## Boundary Conditions

At the outermost interfaces, we apply Neumann boundary conditions:
 - Prescribed heat fluxes at the boundaries ($q_L$ and $q_R$).
 - Setting $q_L$ = $q_R$ = 0.0 models an insulated rod (no heat leaving or entering the domain).

## Explicit Euler Update

A finite-volume update for cell $i$ is:

$$u_i^{n+1} = u_i^n + \Delta t \left( \frac{F_{i} - F_{i+1}}{\Delta x} \right)$$

where:
 - $u_i^n$ is the temperature in cell i at time step n,
 - $\Delta t$ is the time step size,
 - $F_{i}$ and $F_{i+1}$ are the heat fluxes at the left and right interfaces of cell i, respectively.


### Exercise - fill in the missing lines

Complete the following function that advances the temperature by nt steps
using the formulas for the diffusive flux $F$ and the temperature $u$.


In [14]:
def solve_heat_eqn(u, kappa, dx, dt, qL, qR, nt):
    """Numerically solve the 1D heat equation.

    Arguments
    ---------
    u : array,      The initial temperature distribution (C)
    kappa : float,  The thermal diffusivity (m^2/s).
    dx : float,     The spacing between grid points (m).
    dt : float,     The time step size (s).
    qL : float,     The heat flux at the left boundary (W/m^2).
    qR : float,     The heat flux at the right boundary (W/m^2).
    nt : int,       The number of time steps to simulate.

    Returns
    -------
    array,          The updated temperature distribution.
    """

    N = len(u)          # Number of grid points
    F = [0.0] * (N+1)   # Fluxes at interfaces

    for _ in range(nt):

        # Boundary conditions
        F[0] = qL
        F[-1] = qR

        # Fluxes at intermediate interfaces
        for i in range(1, N):
            pass # TODO: F[i]: ...

        # Update the temperature distribution
        for i in range(len(u)):
            pass # TODO: u[i]: ...

    return u
    

**One Possible Solution**

Below collapsed cell includes a possible solution. Compare it with your own implementation.


In [15]:
def solve_heat_eqn(u, kappa, dx, dt, qL, qR, nt):
    """Numerically solve the 1D heat equation.

    Arguments
    ---------
    u : array,      The initial temperature distribution (C)
    kappa : float,  The thermal diffusivity (m^2/s).
    dx : float,     The spacing between grid points (m).
    dt : float,     The time step size (s).
    qL : float,     The heat flux at the left boundary (W/m^2).
    qR : float,     The heat flux at the right boundary (W/m^2).
    nt : int,       The number of time steps to simulate.

    Returns
    -------
    array,          The updated temperature distribution.
    """

    N = len(u)          # Number of grid points
    F = [0.0] * (N+1)   # Fluxes at interfaces

    for _ in range(nt):

        # Boundary conditions
        F[0] = qL
        F[-1] = qR

        # Fluxes at intermediate interfaces
        for i in range(1, N):
            F[i] = -kappa * (u[i] - u[i-1]) / dx

        # Update the temperature distribution
        for i in range(len(u)):
            dudt = (F[i] - F[i+1]) / dx
            u[i] += dt * dudt

    return u

## Quick Sanity Checks

Now, let's try running this function for one timestep with the following input temperature distibution: [0.0, 100.0, 0.0]. The expected output is a new array representing the temperature distribution after one time step. One would expect the middle value, which is greater than its neighbors to decrease given the heat diffusion.

In [17]:
solve_heat_eqn(
    u = [0.0, 100.0, 0.0],
    kappa = 0.1, # thermal diffusivity (m^2/s)
    dx = 1.0,    # spatial step size (m)
    dt = 0.01,    # time step size (s)
    qL = 0.0,    # left boundary heat flux (W/m^2)
    qR = 0.0,    # right boundary heat flux (W/m^2)
    nt = 1
)

[0.1, 99.8, 0.1]

Now, let's re-run the function with longer time steps and see how the temperature distribution evolves.

In [18]:
solve_heat_eqn(
    u = [0.0, 100.0, 0.0],
    kappa = 0.1, # thermal diffusivity (m^2/s)
    dx = 1.0,    # length of each spatial cell (m)
    dt = 1.0,    # time step size (s)
    qL = 0.0,    # left boundary heat flux (W/m^2)
    qR = 0.0,    # right boundary heat flux (W/m^2)
    nt = 1
)

[10.0, 80.0, 10.0]

As expected, the temperature drop is more pronounced with a larger time step. Now let's run the simulation for 1000 time steps instead of just 1.

In [11]:
solve_heat_eqn(
    u = [0.0, 100.0, 0.0],
    kappa = 0.1, # thermal diffusivity (m^2/s)
    dx = 1.0,    # length of each spatial cell (m)
    dt = 0.1,    # time step size (s)
    qL = 0.0,    # left boundary heat flux (W/m^2)
    qR = 0.0,    # right boundary heat flux (W/m^2)
    nt = 1000
)

[33.33333333333137, 33.333333333337265, 33.33333333333137]

The result of the above function call should converge to the steady-state solution of the heat equation, i.e., the mean temperature of the three nodes [0.0, 100.0, 0.0], i.e., 33.33 °C everywhere.

## What we just did

 - **Got something working fast:** based on a scientific description of the problem,
  we quickly implemented a computational solution.

 - **Mixed concerns:** one function that applies flux BCs,
  computes fluxes, and updates temperatures.

 - **Stateful, in-place update:** harder to reason about and to test in isolation.

 - **No reasoning artifacts:** no preconditions/postconditions, no invariants, no stability checks.

 - **Only ad-hoc sanity checks:** nothing automated to guard against regressions.

This "code and fix" pattern is common in scientific computing and fine for 
prototyping, but it doesn’t scale.



## Looking Ahead

Next, we'll:

 - name the behaviors we want (pre/postconditions, invariants like conservation under zero flux),
 - separate concerns (BCs, fluxes, updates), 
 - and refactor stepwise. 

That will make the code testable (Ch.3), let us express properties to try to refute (Ch.4), and even enable bounded checking with contracts (Ch.5).